In [ ]:
import pandas as pd
import numpy as np
from funciones import *

# Configuracion de display de DataFrames en Jupyter
pd.options.display.max_columns = None

# Localizacion del .CSV del dataset
data_location_raw = './datasets/properatti.csv'

# Localizacion del .CSV donde enviaremos los datos limpios
data_location_clean = './datasets/properatti_clean.csv'

# Tipos de datos a importar como categorias
dtypes_cat = {
    "operation": "category",
    "property_type": "category",
    "country_name": "category",
    "state_name": "category",
}

# Columnas del DataFrame Original
dtypes_non_cat_orig = {
    "place_name": "str",
    "description": "str",
    "title": "str",
}

# Columnas del DataFrame limpio
dtypes_non_cat_clean = {
    "price_clean": "float",
    "currency_clean": "category",
    "surface_covered_in_m2_clean": "float",
    "price_per_m2_covered_clean": "float",
}

# Creamos un nuevo diccionario con todos los elementos
dtypes = {}
dtypes.update(dtypes_cat)
dtypes.update(dtypes_non_cat_orig)
dtypes.update(dtypes_non_cat_clean)

# Hacemos la importación del dataframe clean
raw_data = pd.read_csv(data_location_clean,
                       dtype=dtypes,
                       usecols=['Unnamed: 0']+list(dtypes_cat) +
                               list(dtypes_non_cat_orig) +
                               list(dtypes_non_cat_clean),
                       index_col=0
)

# Copiamos el dataframe a otro para tener una copia literal del mismo
data = raw_data.copy()

# Forma de resetear el index del dataframe si no se usa el index_col=0
## data_default.reset_index(drop=True, inplace=True)
## data_str.index

Información del DataFrame RAW que hemos importado

In [ ]:
raw_data.info()

Información del DataFrame que vamos a utilizar para nuestro análisis

In [ ]:
data.info()

Hacemos una impresion inicial del dataframe

In [ ]:
data.head()

In [ ]:
missing_zero_values_table(data)

### Drop de observaciones con NaN
Nos quedaremos unicamente con aquellas observaciones que no tengan datos faltantes.

In [ ]:
data.dropna(inplace=True)
missing_zero_values_table(data)

### Unidades geográficas

Haremos el agrupamiento por unidades geográficas, las cuales estan en state_name.
Primero obtendremos una lista de las unidades geográficas y la cantidad de observaciones 

In [ ]:
list_regiones = data['state_name'].value_counts()
print(list_regiones.to_string())

### Análisis para Capital Federal
Crearemos una máscara para evaluar las observaciones que son exclusivamente de Capital Federal

In [ ]:
mask_capital_federal = data['state_name'] == 'Capital Federal'
print(f"Observaciones en Capital Federal: {mask_capital_federal.sum()}.")

#### Barrios dentro de Capital Federal
Obtenemos una lista de barrios oficiales y no oficiales que hay dentro del set de datos

In [ ]:
print(data[mask_capital_federal]['place_name'].value_counts().to_string())

In [ ]:
listado_barrios = data[mask_capital_federal]['place_name'].unique()
listado_barrios = np.sort(listado_barrios)
print(f"Cantidad de Barrios considerados en Capital Federal: {len(listado_barrios)}.")

In [ ]:
data['property_type'].unique()

In [ ]:
# Verificamos que nos de el valor de un barrio
listado_barrios[0]

In [ ]:
x_data = data.loc[mask_capital_federal & (data['place_name'] == listado_barrios[0]),'price_per_m2_covered_clean'].values
x_data

### Gráfica de Boxplot para detección de outliers por barrio

In [ ]:
# Seleccion del barrio a graficar por indice
barrio_index = 4

y_data = ['PH', 'Apartment', 'House', 'Store']

x0 = data.loc[mask_capital_federal & (data['place_name'] == listado_barrios[barrio_index]) & (data['property_type'] == 'PH'),'price_per_m2_covered_clean'].values
x1 = data.loc[mask_capital_federal & (data['place_name'] == listado_barrios[barrio_index]) & (data['property_type'] == 'apartment'),'price_per_m2_covered_clean'].values
x2 = data.loc[mask_capital_federal & (data['place_name'] == listado_barrios[barrio_index]) & (data['property_type'] == 'house'),'price_per_m2_covered_clean'].values
x3 = data.loc[mask_capital_federal & (data['place_name'] == listado_barrios[barrio_index]) & (data['property_type'] == 'store'),'price_per_m2_covered_clean'].values

x_data = [x0, x1, x2, x3]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
          'rgba(255, 65, 54, 0.5)']

import plotly.graph_objects as go

fig = go.Figure()

for xd, yd, cls in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            x=xd,
            name=yd,
            boxpoints='suspectedoutliers',
            jitter=0.5,
            pointpos=0,
            fillcolor=cls,
            whiskerwidth=0.7,
            line_width=1)
        )


fig.update_yaxes(type='category')


fig.update_layout(
    title=
    f'<b>Boxplot por tipo de propiedad</b>'+
    f'<br>{listado_barrios[barrio_index]}',
    #title=f"Boxplot por tipo de propiedad: {listado_barrios[barrio_index]}",
    #subtitle="By Class of Car",
    xaxis=dict(
        title='Precio por metro<sup>2</sup> cubierto (<b>USD</b>)',
        autorange=True,
        showgrid=True,
        zeroline=False,
        ticks="outside",
        gridwidth=0.5
#        nticks=15
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=True,
    #hovermode='x unified'
)

fig.update_traces(hovertemplate="Precio: %{x:.0f} / m<sup>2</sup>")
fig.show()

fig.write_image("images/fig1.svg")
